## 1 - Подготовка данных
**Цель:**
Провести первичную подготовку данных для последующего построения моделей, включая формирование целевой переменной, фильтрацию данных, обработку качественных и количественных признаков, а также интеграцию модулей группы и государственной поддержки.

**Prerequisites:**

- Наличие корпоративного портфеля;
- Данные о рейтингах, дефолтах и здоровом портфеле;
- Конфигурационный файл CONFIGS.yaml с параметрами предобработки.

**Результат:**

Подготовленный набор данных, включая:
- Целевую переменную target (дефолт) и флаг survive;
- Обработанные качественные и количественные признаки;
- Данные, отфильтрованные по периоду наблюдения и сегменту C+;
- Готовые датафреймы для дальнейшего моделирования.

-----------

### 1.1 - Библиотеки

In [1]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

import yaml

import warnings
warnings.filterwarnings("ignore")

import plotly.express as px

from sklearn import preprocessing
import yaml
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm

# from funcs_dataset_preparation import * 
from _utils.qualitative_features_analysis import *
from _utils.text_preporation import text_preprocessing, get_missing_values


In [2]:
pd.set_option('display.max_columns', None)

### 1.2 - Настройка конфигурации

In [3]:
with open('../CONFIGS.yaml', 'r') as file:
    CONFIG = yaml.safe_load(file)

PATHS = CONFIG['data_paths']

In [4]:
pp_params = CONFIG['preprocessing_params']

observation_period_days = pp_params['observation_period_days']

observation_period =  (  pd.to_datetime(pp_params['observation_period'][0], format='%Y-%m-%d')
                       , pd.to_datetime(pp_params['observation_period'][1], format='%Y-%m-%d'))

placeholder_date = pp_params['placeholder_date']
features_to_drop = pp_params['features_to_drop']
C_plus_segments = pp_params['C_plus_segments']

module_cols = {
      'young_companies' : CONFIG['features']['young_companies']
    , 'gov_support'     : CONFIG['features']['gov_support']
    , 'group_support'   : CONFIG['features']['group_support']
    , 'warnings'        : CONFIG['features']['warnings']
    
}

valid_date_comparison = CONFIG['preprocessing_params']['valid_date_comparison'] # year before right threshold  

In [5]:
def show_example(data, example_client=706021):
    return data.query('client_id == @example_client')

### 1.3 - Загрузка данных

In [6]:
filepaths = {
      'ratings_compact'       : PATHS['ratings']['preprocessed_compact']
    , 'ratings_full'          : PATHS['ratings']['preprocessed_full']
    , 'defaults_preprocessed' : PATHS['risk']['defaults_preprocessed']
    , 'healthy_portfolio'     : PATHS['portfolio']['healthy']
    , 'warnings'       : PATHS['module_warnings']['raw_data']
    , 'qualitative_mapping_original' : PATHS['module_qualitative']['mapping_original']
    , 'clean_data'    : PATHS['other']['clean_data']
    , 'qualitative_df' : PATHS['module_qualitative']['processed_data']
    , 'quantitative_df': PATHS['module_quantitative']['processed_data']
    , 'group_df_preprocessed' : PATHS['module_group']['processed_data']
    , 'group_df_influence'    : PATHS['module_group']['influence']
    , 'gov_df_preprocessed' : PATHS['module_gov']['processed_data']
    , 'gov_df_influence'    : PATHS['module_gov']['influence']
    , 'gov_mapping' : PATHS['module_gov']['mapping_original']
    , 'warnings_df' : PATHS['module_warnings']['processed_data']
    , 'youngs_df'   : PATHS['case_youngs']['processed_data']
    , 'valid_dates' : PATHS['ratings']['valid_dates']
}

In [7]:
raw_data = {}

# Рейтинги
raw_data['ratings'] = pd.read_csv(filepaths['ratings_compact'], encoding='cp1251', sep='|')
raw_data['ratings']['rating_assignment_date'] = pd.to_datetime(raw_data['ratings']['rating_assignment_date'], format='%Y-%m-%d')

raw_data['ratings_full'] = pd.read_csv(filepaths['ratings_full'], encoding='utf-8', sep='|')
raw_data['ratings_full']['rating_assignment_date'] = pd.to_datetime(raw_data['ratings_full']['rating_assignment_date'], format='%Y-%m-%d')

# Дефолты
raw_data['defaults'] = pd.read_csv(filepaths['defaults_preprocessed'], encoding='cp1251', sep='|')

# Здоровый портфель
raw_data['healthy_portfolio'] = pd.read_csv(filepaths['healthy_portfolio'])

for date_col in ['report_date', 'date', 'segment_first_entry_date']:
    raw_data['healthy_portfolio'][date_col] = pd.to_datetime(raw_data['healthy_portfolio'][date_col], format='%Y-%m-%d')

# Предупреждающие сигналы
# raw_data['warnings'] = pd.read_csv(filepaths['warnings'], delimiter='\t')
raw_data['warnings'] = pd.read_excel(filepaths['warnings'])

# Загрузка маппинга для качественных переменных
with open(filepaths['qualitative_mapping_original'], 'r', encoding='utf-8') as stream:
    raw_data['qualitative_mapping'] = yaml.safe_load(stream)

# Загрузка маппинга для качественных переменных
with open(filepaths['gov_mapping'], 'r', encoding='utf-8') as stream:
    raw_data['gov_mapping'] = yaml.safe_load(stream)

for k in ['ratings', 'ratings_full', 'defaults', 'healthy_portfolio', 'warnings']:
    try:
        raw_data[k].drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')
    except:
        pass

In [8]:
print("\n📋 Загруженные датасеты:")
for name, df in raw_data.items():
    if isinstance(df, pd.DataFrame):
        print(f"  - {name}: {df.shape}")


📋 Загруженные датасеты:
  - ratings: (9139, 6)
  - ratings_full: (9139, 279)
  - defaults: (297, 3)
  - healthy_portfolio: (30459, 9)
  - warnings: (42, 5)


In [9]:
raw_data['ratings_full'][['e9', 'e10', 'e12']] = raw_data['ratings_full'][['e9', 'e10', 'e12']].astype(float, errors='ignore')

### 1.4 - Подготовка основного датафрейма

In [10]:
print("\n🔧 Подготовка основного датафрейма...")

# Выбираем нужные колонки и переименовываем их
main_df = raw_data['ratings'].copy()

print(f"Основной датафрейм: {main_df.shape}")
print("\nПервые 3 записи:")
display(main_df.head(3))

# %%
# Проверка типов данных
print("Типы данных:")
main_df.dtypes


🔧 Подготовка основного датафрейма...
Основной датафрейм: (9139, 6)

Первые 3 записи:


,bin,client_id,rating_id,rating_preliminary,rating_final,rating_assignment_date
0,40940002520,781908,0,14,14.0,2018-08-08
1,90240019261,749596,1,13,13.0,2018-08-08
2,911040000150,711998,3,13,13.0,2018-02-14


Типы данных:


bin                                int64
client_id                          int64
rating_id                          int64
rating_preliminary                 int64
rating_final                     float64
rating_assignment_date    datetime64[ns]
dtype: object

### 1.5 - Создание целевой переменной

In [11]:
def create_target_variable(ratings_df: pd.DataFrame, 
                           defaults_df: pd.DataFrame,
                           params: dict) -> pd.DataFrame:
    """
    Что делает: 
        - Объединяет данные о рейтингах с информацией о дефолтах
        - Создает бинарную переменную target (1 = дефолт, 0 = нет дефолта)
        - Рассчитывает флаг выживания клиента
    """
    print("\n🎯 Формирование целевой переменной...")
    
    # Объединяем рейтинги с данными о дефолтах
    result_df = pd.merge(
        ratings_df,
        defaults_df,
        how='left',
        on=['client_id'],
        indicator=True
    )
    
    # Заполняем отсутствующие даты дефолта далекой будущей датой
    # (для клиентов, которые не попали в дефолт)
    result_df['default_date'] = result_df['default_date'].fillna(params['placeholder_date'])
    result_df['default_date'] = pd.to_datetime(result_df['default_date'])
    
    result_df['recovery_date'] = result_df['recovery_date'].fillna(params['placeholder_date'])
    result_df['recovery_date'] = pd.to_datetime(result_df['recovery_date'])
    
    # Преобразуем даты к единому формату
    result_df['rating_assignment_date'] = pd.to_datetime(result_df['rating_assignment_date'])
    
    # Рассчитываем конечную дату периода наблюдения
    result_df['observation_end_date'] = (
        result_df['rating_assignment_date'] + pd.Timedelta(days=params['observation_period_days'])
    )
    
    # Создаем целевую переменную
    # target = 1, если дефолт произошел в течение периода наблюдения
    default_within_period = result_df['default_date'] <= result_df['observation_end_date']
    default_in_the_future = result_df['default_date'] >= result_df['rating_assignment_date']
    result_df['target'] = np.where(default_within_period & default_in_the_future, 1, 0)

    default_in_the_past    = result_df['default_date']  <= result_df['rating_assignment_date']  
    recovery_in_the_future = result_df['recovery_date'] >= result_df['rating_assignment_date']
    result_df['survive'] = np.where(default_in_the_past & recovery_in_the_future, 0, 1)
    
    print(f"✅ Создана целевая переменная. Дефолтов: {result_df['target'].sum()} из {len(result_df)}")
    print(f"   Уровень дефолта: {result_df['target'].mean():.2%}")
    print(f"   Уровень выживания: {result_df['survive'].mean():.2%}")
    
    return result_df

In [12]:
# Формируем целевую переменную на основе дефолтов
target_df = create_target_variable(
    ratings_df=main_df,
    defaults_df=raw_data['defaults'],
    params=pp_params
)


🎯 Формирование целевой переменной...
✅ Создана целевая переменная. Дефолтов: 632 из 9630
   Уровень дефолта: 6.56%
   Уровень выживания: 93.09%


In [13]:
# # %%
# # Визуализация распределения целевой переменной

# target_dist = target_df['target'].value_counts().reset_index()
# target_dist.columns = ['target', 'count']
# target_dist['target'] = target_dist['target'].map({0: 'Не дефолт', 1: 'Дефолт'})

# fig = px.pie(target_dist, values='count', names='target', 
#              title='Распределение целевой переменной',
#              color_discrete_map={'Не дефолт': '#2E7D32', 'Дефолт': '#C62828'})
# fig.show()

In [14]:
aggreg_target_df = target_df\
                        .groupby(['rating_id', 'client_id'], as_index = False)\
                        .agg({  'target' : 'max'
                              , 'survive': 'min'})\
                        .rename(columns={  'target'  : 'target'
                                          , 'survive' : "survive"
                                          })
aggreg_target_df.head()

,rating_id,client_id,target,survive
0,0,781908,0,1
1,1,749596,0,1
2,3,711998,0,1
3,4,2662074,0,1
4,5,748902,0,1


In [15]:
c_plus_segment_df =  raw_data['healthy_portfolio']\
                        .query('segment.isin(@C_plus_segments)', engine='python').copy()\
                        [['row_id', 'client_id', 'date', 'segment', 'segment_first_entry_date']]

In [16]:
c_plus_segment_df

,row_id,client_id,date,segment,segment_first_entry_date
0,0,176783,2018-02-01,C,2018-02-01
1,1,176783,2018-03-01,C,2018-02-01
2,2,176783,2018-04-01,C,2018-02-01
3,3,176783,2018-05-01,C,2018-02-01
4,4,176783,2018-06-01,C,2018-02-01
...,...,...,...,...,...
30454,491495,7497819,2025-08-01,C,2025-08-01
30455,491499,7500722,2025-07-01,C,2025-07-01
30456,491500,7500722,2025-08-01,C,2025-07-01
30457,491501,7520054,2025-08-01,C,2025-08-01


In [17]:
aggreg_c_plus_segment_df = c_plus_segment_df\
                            .groupby('client_id')['date']\
                            .min()\
                            .reset_index()\
                            .rename(columns={'date':'segment_first_entry_date_when_healthy'})
                            
aggreg_c_plus_segment_df

,client_id,segment_first_entry_date_when_healthy
0,176783,2018-02-01
1,197185,2018-02-01
2,197228,2018-02-01
3,197255,2018-07-01
4,197347,2019-03-01
...,...,...
1187,7451551,2025-06-01
1188,7497819,2025-08-01
1189,7500722,2025-07-01
1190,7520054,2025-08-01


In [18]:
healthy_ratings_df = pd.merge(main_df,
                      c_plus_segment_df,
                      how='left',
                      on=['client_id'])

In [19]:
healthy_ratings_df['date'] = pd.to_datetime(healthy_ratings_df['date'])

record_within_half_a_year = ((healthy_ratings_df['date'] - healthy_ratings_df['rating_assignment_date']).dt.days < 183) & \
                            ((healthy_ratings_df['date'] - healthy_ratings_df['rating_assignment_date']).dt.days >= 0)
healthy_ratings_df['in_portfolio'] = record_within_half_a_year.astype(int)


In [20]:
healthy_ratings_df['in_portfolio'].value_counts(dropna=False)

in_portfolio
0    256546
1     31139
Name: count, dtype: int64

In [21]:
aggreg_healthy_ratings_df = healthy_ratings_df\
                            .groupby(['rating_id', 'client_id'], as_index = False)\
                            .agg({'in_portfolio': 'max'})
aggreg_healthy_ratings_df.head()

,rating_id,client_id,in_portfolio
0,0,781908,0
1,1,749596,0
2,3,711998,1
3,4,2662074,1
4,5,748902,1


In [22]:
# show_example(aggreg_healthy_ratings_df)

In [23]:
main_df = main_df.merge(aggreg_target_df         , how='left', on=['client_id', 'rating_id'])\
                 .merge(aggreg_healthy_ratings_df, how='left', on=['client_id', 'rating_id'])\
                 .merge(aggreg_c_plus_segment_df , how='left', on='client_id')

In [24]:
main_df['is_segment'] = np.where((main_df['rating_assignment_date'] >= main_df['segment_first_entry_date_when_healthy']) &\
                             (main_df['segment_first_entry_date_when_healthy'].isna() == False), 1, 0)

main_df['rating_assignment_date'] = pd.to_datetime(main_df['rating_assignment_date'])
main_df['drop_date'] = pd.to_datetime(main_df['rating_assignment_date']).dt.date


In [25]:
main_df

,bin,client_id,rating_id,rating_preliminary,rating_final,rating_assignment_date,target,survive,in_portfolio,segment_first_entry_date_when_healthy,is_segment,drop_date
0,40940002520,781908,0,14,14.0,2018-08-08,0,1,0,2021-02-01,0,2018-08-08
1,90240019261,749596,1,13,13.0,2018-08-08,0,1,0,NaT,0,2018-08-08
2,911040000150,711998,3,13,13.0,2018-02-14,0,1,1,2018-02-01,1,2018-02-14
3,890619300019,2662074,4,13,13.0,2018-02-14,0,1,1,2018-02-01,1,2018-02-14
4,991240000043,748902,5,14,14.0,2018-01-31,0,1,1,2018-02-01,0,2018-01-31
...,...,...,...,...,...,...,...,...,...,...,...,...
9134,170240026261,5776650,10913,16,16.0,2024-05-31,0,1,1,2021-04-01,1,2024-05-31
9135,811109350398,714412,10914,18,18.0,2024-06-11,0,1,1,2023-06-01,1,2024-06-11
9136,111040002774,2090266,10918,16,16.0,2024-05-06,0,1,0,2021-08-01,1,2024-05-06
9137,200340024683,5850003,10919,17,17.0,2024-05-06,0,1,1,2021-06-01,1,2024-05-06


### 1.6 - Фильтрация данных

In [26]:
filtered_df = main_df.copy()

In [27]:
def apply_1st_level_filtration(df: pd.DataFrame) -> pd.DataFrame:
    print("\n🔍 1/2 Фильтрация данных...")

    print('\t Без фильтрации:', df.shape[0])

    df.sort_values(['client_id','rating_assignment_date'], inplace=True)

    df = df.query('rating_final.isna() == False', engine = 'python')
    print('\t После отсечения пустых рейтингов:', df.shape[0])

    df = df.drop_duplicates(subset=['client_id', 'drop_date'], keep = 'last')
    print('\t После удаления дубликатов рейтингов:', df.shape[0])

    df = df.query('is_segment == 1')
    print('\t После удаления не в сегменте рейтингов:', df.shape[0])

    return df

In [28]:
filtered_df = apply_1st_level_filtration(filtered_df)


🔍 1/2 Фильтрация данных...
	 Без фильтрации: 9139
	 После отсечения пустых рейтингов: 9137
	 После удаления дубликатов рейтингов: 8792
	 После удаления не в сегменте рейтингов: 6369


In [29]:
# valid_date_comparison = '2023-07-01' # year before right threshold 

In [30]:
# Расчет валидности рейтинга. Рейтинг может пересчитываться в течение года несколько раз.
# В целом существует проблема неоднородности данных: некоторые рейтинги пересчитывались чаще, чем другие.
# Рейтинг не валиден, если в последний раз он пересчитывался более года назад.
# Также, при попадании под особое правило для недавних рейтингов, которые были рассчитаны меньше года назад.
def get_valid_dates(df):
    df = df[['client_id', 'rating_id', 'rating_assignment_date']]
    valid_dates = df.sort_values(['client_id', 'rating_assignment_date']).reset_index(drop=True)

    
    # Находим дату перерасчета рейтинга (наличие след. рейтинга после него, дата след. рейтинга и будет крайней датой валидности текущего рейтинга)
    valid_dates['valid_date'] = valid_dates.groupby('client_id')['rating_assignment_date'].transform(
        lambda x: x.shift(-1)
    )

    
    # Если следующий рейтинг отсутствует, то заполняем датой рейтинга + год, т.к. рейтинг не должен быть старше года.
    mask_na = valid_dates['valid_date'].isna()
    valid_dates.loc[mask_na, 'valid_date'] = \
        valid_dates.loc[mask_na, 'rating_assignment_date'] + pd.Timedelta(days=365)

    
    # Ограничение 365 дней: рейтинг не должен быть старше года.
    days_diff = (valid_dates['valid_date'] - valid_dates['rating_assignment_date']).dt.days
    mask_too_long = days_diff > 365
    valid_dates.loc[mask_too_long, 'valid_date'] = \
        valid_dates.loc[mask_too_long, 'rating_assignment_date'] + pd.Timedelta(days=365)

    
    # Особое правило для рейтингов рассчитанных год назад от даты разработки: они еще не вызрели.
    mask_recent = valid_dates['rating_assignment_date'] >= valid_date_comparison
    valid_dates.loc[mask_recent, 'valid_date'] = \
        valid_dates.loc[mask_recent, 'rating_assignment_date'] + pd.Timedelta(days=365)
    
    return valid_dates[['client_id', 'rating_id', 'rating_assignment_date', 'valid_date']]

In [31]:
vd = get_valid_dates(show_example(filtered_df))

In [32]:
valid_dates = get_valid_dates(filtered_df)

In [33]:
def apply_2nd_level_filtration(df: pd.DataFrame, start_date, end_date) -> pd.DataFrame:
    print("\n🔍 2/2 Фильтрация данных...")

    print('\t Без фильтрации:', df.shape[0])

    df = df.query('rating_final != 26')
    print('\t После исключения 26 рейтингов:', df.shape[0])

    df = df.query('in_portfolio == 1')
    print('\t После пересечения с здоровым портфелем:', df.shape[0])

    df = df.query('survive == 1')
    print('\t После отсечения рейтингов в период дефолта:', df.shape[0])

    df = df.query('rating_assignment_date >= @start_date and rating_assignment_date < @end_date')
    print('\t После отсечения по времени:', df.shape[0])

    return df

In [34]:
filtered_df = apply_2nd_level_filtration(df=filtered_df, start_date=observation_period[0], end_date=observation_period[1])


🔍 2/2 Фильтрация данных...
	 Без фильтрации: 6369
	 После исключения 26 рейтингов: 6049
	 После пересечения с здоровым портфелем: 5154
	 После отсечения рейтингов в период дефолта: 5152
	 После отсечения по времени: 4985


In [35]:
filtered_df.target.sum(), filtered_df.target.shape[0], filtered_df.target.sum()/filtered_df.target.shape[0]*100

(np.int64(249), 4985, np.float64(4.994984954864594))

### 1.7 - Подгтовока качественных признаков 

In [36]:
print("\n📝 Обработка качественных признаков...")

used_columns=[
    'rating_id', 
    'client_id',
    'rating_final', 
    'rating_preliminary',
    'rating_assignment_date'
]
# какие колонки являются качественными
qualitative_columns = []
for col in raw_data['ratings_full'].columns:
    if raw_data['ratings_full'][col].dtype == 'object' and col not in used_columns:
        qualitative_columns.append(col)

print(f"Найдено качественных признаков: {len(qualitative_columns)}")


📝 Обработка качественных признаков...
Найдено качественных признаков: 121


In [37]:
too_many_values = raw_data['ratings_full'][qualitative_columns]\
    .nunique()\
    .sort_values(ascending=False)

too_many_values = list(too_many_values[too_many_values>=10].index)

In [38]:
drop_cols = ['status', 'integration_id'] + too_many_values
# drop_cols = ['status', 'integration_id', 't', 's1', 's3', 'sss']

In [39]:
id_cols = ['client_id', 'rating_id']

In [40]:
qualitative_columns = [ col for col in qualitative_columns if col not in id_cols + drop_cols]

In [41]:
qualitative_columns = [\
    'g1',
    'g2',
    'g3',
    'g4',
    'g5',
    'g6',
    'g7',
    'g8',
    'g9',
    'g10',
    'g11',
    'g12',
    'g13',
    'g14',
    'g15',
    'g16',
    'g17',
    'g18',
    'g19',
    'g20',
    'g21',
    'g22',
    'g23',
    'g24',
    'g77'
 ]

In [42]:
if qualitative_columns:

    qual_df = pd.merge(
        filtered_df[id_cols + ['rating_assignment_date', 'target']],
        raw_data['ratings_full'][id_cols + qualitative_columns],
        on=id_cols,
        how='left'  # или 'left' если нужно сохранить все записи из target_df
    )

    qual_df_processed = qual_df.copy()

    for col in tqdm(qualitative_columns, desc="Обработка текста"):
        qual_df_processed[col] = qual_df_processed[col].apply(lambda x: text_preprocessing(str(x)))

    qualitative_columns_renamed= {col: f'kfx{i}' for i, col in enumerate(qualitative_columns)}
    qual_df_processed.rename(columns=qualitative_columns_renamed, inplace=True)
    
    # print_statistics(qual_df_processed, "Качественные признаки")
else:
    qual_df_processed = filtered_df[id_cols + ['rating_assignment_date', 'target']].copy()

Обработка текста:   0%|          | 0/25 [00:00<?, ?it/s]

In [43]:
print(features_to_drop)

['kfx17', 'kfx24', 'kfx21']


In [44]:
qual_df_processed.drop(features_to_drop,axis=1,inplace=True)

In [45]:
qualitative_mapping = raw_data['qualitative_mapping']

In [46]:
qualitative_columns = list(qualitative_columns_renamed.values())

In [47]:
for col_name in qualitative_columns: 
    if col_name in qual_df_processed.columns:
        qual_df_processed[col_name] = qual_df_processed[col_name].map(qualitative_mapping[col_name])
    else: 
        print(f'Column {col_name} нет в датафрейме')

Column kfx17 нет в датафрейме
Column kfx21 нет в датафрейме
Column kfx24 нет в датафрейме


In [48]:
qual_df_processed.isna().sum()

client_id                 0
rating_id                 0
rating_assignment_date    0
target                    0
kfx0                      0
kfx1                      0
kfx2                      0
kfx3                      0
kfx4                      0
kfx5                      0
kfx6                      0
kfx7                      0
kfx8                      0
kfx9                      0
kfx10                     0
kfx11                     0
kfx12                     0
kfx13                     1
kfx14                     0
kfx15                     0
kfx16                     0
kfx18                     0
kfx19                     0
kfx20                     0
kfx22                     0
kfx23                     0
dtype: int64

In [49]:
qual_df_processed.dropna(inplace=True)

#### 1.7.1 - Подход 1: Анализ на уровне рейтингов

In [50]:
# Анализируем
ratings_level_analysis_results = comprehensive_feature_analysis(
    df=qual_df_processed,
    features=qualitative_columns,
    target_col='target'
)

# Получаем рекомендации
features_to_drop_rating_level = recommend_features_to_drop(
    ratings_level_analysis_results,
    iv_threshold=0.02,
    variance_threshold=0.90
)

Анализ признака: kfx0
Анализ признака: kfx1
Анализ признака: kfx2
Анализ признака: kfx3
Анализ признака: kfx4
Анализ признака: kfx5
Анализ признака: kfx6
Анализ признака: kfx7
Анализ признака: kfx8
Анализ признака: kfx9
Анализ признака: kfx10
Анализ признака: kfx11
Анализ признака: kfx12
Анализ признака: kfx13
Анализ признака: kfx14
Анализ признака: kfx15
Анализ признака: kfx16
Анализ признака: kfx18
Анализ признака: kfx19
Анализ признака: kfx20
Анализ признака: kfx22
Анализ признака: kfx23

📊 Рекомендовано к удалению 6 признаков из 22:

  ❌ kfx1: IV=0.0155 < 0.02; Dominant freq=91.97% > 90%
  ❌ kfx2: IV=0.0187 < 0.02; Dominant freq=97.11% > 90%
  ❌ kfx11: Dominant freq=93.08% > 90%
  ❌ kfx12: Dominant freq=93.76% > 90%
  ❌ kfx16: IV=0.0005 < 0.02; Dominant freq=97.15% > 90%
  ❌ kfx22: IV=0.0008 < 0.02; Dominant freq=90.73% > 90%


#### 1.7.2 - Подход 2: Анализ на уровне клиентов 

In [51]:
# Берем только последний рейтинг для каждого клиента
latest_ratings = qual_df_processed.sort_values('rating_assignment_date').groupby('client_id').last().reset_index()

# Анализируем
qualitative_columns = [col for col in latest_ratings.columns if col.startswith('kfx')]
clients_level_analysis_results = comprehensive_feature_analysis(
    df=latest_ratings,
    features=qualitative_columns, 
    target_col='target'
)

# Получаем рекомендации
features_to_drop_clients_level = recommend_features_to_drop(
    clients_level_analysis_results,
    iv_threshold=0.02,
    variance_threshold=0.9
)

Анализ признака: kfx0
Анализ признака: kfx1
Анализ признака: kfx2
Анализ признака: kfx3
Анализ признака: kfx4
Анализ признака: kfx5
Анализ признака: kfx6
Анализ признака: kfx7
Анализ признака: kfx8
Анализ признака: kfx9
Анализ признака: kfx10
Анализ признака: kfx11
Анализ признака: kfx12
Анализ признака: kfx13
Анализ признака: kfx14
Анализ признака: kfx15
Анализ признака: kfx16
Анализ признака: kfx18
Анализ признака: kfx19
Анализ признака: kfx20
Анализ признака: kfx22
Анализ признака: kfx23

📊 Рекомендовано к удалению 7 признаков из 22:

  ❌ kfx1: Dominant freq=91.60% > 90%
  ❌ kfx2: Dominant freq=96.83% > 90%
  ❌ kfx11: Dominant freq=93.18% > 90%
  ❌ kfx12: Dominant freq=93.79% > 90%
  ❌ kfx16: IV=0.0024 < 0.02; Dominant freq=97.08% > 90%
  ❌ kfx18: Dominant freq=90.62% > 90%
  ❌ kfx22: IV=0.0141 < 0.02; Dominant freq=91.96% > 90%


In [52]:
features_to_drop_intersection = list(np.sort(list(set(features_to_drop_clients_level).intersection(set(features_to_drop_rating_level)))))
print(len(features_to_drop_intersection))

6


In [53]:
features_to_drop_combination = list(np.sort(list(set(features_to_drop_clients_level + features_to_drop_rating_level))))
print(len(features_to_drop_combination))

7


In [54]:
# features_to_keep = [col for col in qualitative_columns if col not in features_to_drop_intersection]
features_to_keep = [col for col in qualitative_columns if col not in features_to_drop_rating_level]

In [55]:
features_to_drop_rating_level

['kfx1', 'kfx2', 'kfx11', 'kfx12', 'kfx16', 'kfx22']

In [56]:
qual_df_final = qual_df_processed[id_cols + features_to_keep].copy()

In [57]:
qual_df_final

,client_id,rating_id,kfx0,kfx3,kfx4,kfx5,kfx6,kfx7,kfx8,kfx9,kfx10,kfx13,kfx14,kfx15,kfx18,kfx19,kfx20,kfx23
0,176783,1737,2,2,2,4,3,1,4,3,3,3.0,2,5,2,1,1,1
1,197185,1048,3,1,1,3,4,1,5,2,2,2.0,4,5,3,2,7,1
2,197255,912,2,2,2,3,2,2,5,2,3,2.0,2,5,2,2,7,2
3,197255,1734,2,2,2,3,2,2,5,2,3,2.0,2,5,2,2,7,2
4,197255,2254,2,2,2,2,2,2,5,2,3,2.0,2,5,2,2,7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4980,6515504,10692,2,1,1,2,2,1,1,1,4,3.0,1,5,2,1,2,1
4981,6601130,10455,2,1,2,3,3,1,1,1,1,2.0,1,4,2,1,9,1
4982,6610199,10122,3,2,1,4,4,2,5,2,3,3.0,2,5,2,3,4,1
4983,6610199,10839,3,2,1,4,4,2,5,2,3,3.0,2,5,2,3,9,1


### 1.8 - Подготовка количественных признаков 

In [58]:
# quantitive_cols = raw_data['ratings_full'].select_dtypes(np.number).columns[6:].tolist()
quantitive_cols = raw_data['ratings_full'].select_dtypes(np.number).columns[6:].tolist()

In [59]:
quantitive_cols = [col for col in quantitive_cols
                    if col not in module_cols['young_companies']+list(module_cols['gov_support'].keys())+list(module_cols['group_support'].keys())+module_cols['warnings']
                    and 'date' not in col]

In [60]:
# quant_df = raw_data['ratings_full'][id_cols+quantitive_cols].copy()
quant_df = filtered_df[id_cols]\
                .merge(  raw_data['ratings_full'][id_cols+quantitive_cols]
                       , on=id_cols
                       , how='left')

In [61]:
def rename_quantitative_columns(quant_df, quantitative_features):
    """
    Переименование колонок количественных признаков.
    Точно как в старом ноутбуке - ячейка 35.
    """

    renaming_done = 0

    cols = []
    for i in quantitative_features:
        cols.append(i[-3:])  # Берем последние 3 символа
        print(i)
        print(i[-3:])
        print(cols)
        print(' ')
    
    for i in range(len(cols)):
        cols[i] = cols[i].replace(' ', '')
        if cols[i].startswith('te'):
            cols[i] = 'da' + cols[i]
            renaming_done += 1
        elif cols[i] == '3_1':
            cols[i] = 'd1' + cols[i]
            renaming_done += 1
        elif cols[i] == '0_1':
            cols[i] = 'd2' + cols[i]
            renaming_done += 1
        elif cols[i] == 'ear':
            cols[i] = 'd18_year'
            renaming_done += 1
    
    # Применяем новые имена к колонкам
    for i in range(len(quant_df.columns[4:])):
        quant_df.columns.values[4 + i] = cols[i]

    print(renaming_done)
    
    return quant_df

In [62]:
quant_df_size = quant_df.shape[0]

In [63]:
quant_df_missing_count = quant_df[quantitive_cols].isna().sum().sort_values(ascending=False)
quant_df_missing_pct   = quant_df_missing_count\
                            .div(quant_df_size)\
                            .multiply(100)\
                            .round(2)
quant_df_missing_pct

g99    100.00
d10    100.00
c26    100.00
c17     94.78
c19     94.78
        ...  
e28      0.00
e30      0.00
e31      0.00
e32      0.00
e33      0.00
Length: 143, dtype: float64

In [64]:
[i for i in quant_df_missing_pct.index if 'd' in i]

['d10',
 'd20_1',
 'd13_1',
 'd26',
 'd12',
 'd16',
 'd19',
 'd20',
 'd13',
 'd17',
 'd21',
 'd24',
 'd23',
 'd22',
 'd14',
 'd25',
 'd11',
 'd8',
 'd9',
 'd6',
 'd7',
 'd5',
 'd4',
 'd2',
 'd3',
 'd1',
 'd18',
 'd15']

In [65]:
quant_df_missing_pct.loc[[i for i in quant_df_missing_pct.index if 'd' in i]]

d10      100.00
d20_1     94.78
d13_1     94.78
d26        5.24
d12        5.22
d16        5.22
d19        5.22
d20        5.22
d13        5.22
d17        5.22
d21        5.22
d24        5.22
d23        5.22
d22        5.22
d14        4.83
d25        4.83
d11        0.04
d8         0.00
d9         0.00
d6         0.00
d7         0.00
d5         0.00
d4         0.00
d2         0.00
d3         0.00
d1         0.00
d18        0.00
d15        0.00
dtype: float64

In [66]:
less_than_90_pct_missing = \
                [ col 
                    for col in quant_df_missing_pct[quant_df_missing_pct<90].index.tolist()
                    if col not in id_cols 
                ] 

In [67]:
quant_df_processed = quant_df[id_cols + less_than_90_pct_missing]

In [68]:
hyp_df = quant_df_processed.merge(target_df[id_cols+['rating_assignment_date']], how='inner', on=id_cols)
# hyp_df = quant_df.merge(target_df[id_cols+['rating_assignment_date']], how='inner', on=id_cols)
# hyp_df = quant_df_processed.copy()

# создаем столбец для месяца года
hyp_df['year_month'] = hyp_df['rating_assignment_date'].dt.strftime('%Y-%m').astype(str)

cols_to_drop = id_cols + ['rating_assignment_date', 'year_month']

gp_hyp_df = hyp_df.groupby("year_month").apply(lambda x: np.round(x.isnull().sum()/x.shape[0] * 100)).drop(columns=cols_to_drop)
gp_hyp_df

,h9,d26,h13,h3,h10,h5,h4,h11,h7,h8,h12,d12,h6,d16,h16,h15,h14,h2,d19,d20,d13,d17,h1,d21,d24,d23,d22,d14,d25,a10,d11,c5,a9,a11,a12,a3,a6,a5,a4,a1,a2,a8,a7,a15,a14,a13,a16,a20,a21,a18,a17,a24,a25,a26,b1,b2,b3,a22,a19,d8,d9,d6,d7,d5,d4,d2,d3,d1,c7,c8,c11,c6,c9,b4,a23,b5,b6,b8,b7,b10,b11,b12,b9,b13,b14,b16,b15,c1,c2,c3,c4,d18,d15,e2,e1,e4,e5,e6,e3,e8,e11,e13,e14,e15,e16,e17,e7,e18,e19,e21,e20,e23,e24,e25,e22,e26,e27,e29,e28,e30,e31,e32,e33
year_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-07,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [69]:
import plotly.express as px

# Тепловая карта пропусков по месяцам и признакам
fig = px.imshow(
    gp_hyp_df.T,  # Транспонируем: месяцы по X, признаки по Y
    x=gp_hyp_df.index,  # Месяцы на оси X
    y=gp_hyp_df.columns,  # Признаки на оси Y
    color_continuous_scale='Reds',
    title='Доля пропущенных значений по месяцам (%)',
    labels={'x': 'Месяц', 'y': 'Признаки', 'color': '% пропусков'}
)

fig.update_layout(
    width=1400,  # Увеличенная ширина
    height=800,  # Увеличенная высота
    xaxis_tickangle=-45,
    font=dict(size=16),  # Размер шрифта для всех элементов
    title_font_size=20,  # Размер шрифта заголовка
    title_x=0.5,  # Центрирование заголовка
    xaxis=dict(tickfont=dict(size=16)),  # Размер шрифта оси X
    yaxis=dict(tickfont=dict(size=16)),  # Размер шрифта оси Y
    coloraxis_colorbar=dict(
        title_font_size=16,  # Размер шрифта заголовка цветовой шкалы
        tickfont=dict(size=16)  # Размер шрифта меток цветовой шкалы
    )
)
fig.show()

In [70]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        y = gp_hyp_df.columns,
        x = gp_hyp_df.index,
        orientation='h'
    )
)
fig.show()

### 1.9 - Молодые компании

In [71]:
module_cols['young_companies']

['f1', 'f2', 'f3', 'f4', 'f5']

In [72]:
youngs_df = filtered_df[id_cols]\
                .merge(  raw_data['ratings_full'][id_cols+module_cols['young_companies']]
                       , on=id_cols
                       , how='left')

In [73]:
# youngs_df = [id_cols+ module_cols['young_companies']]
youngs_df

,client_id,rating_id,f1,f2,f3,f4,f5
0,176783,1737,0.0,1.0,2.0,4.0,6.0
1,197185,1048,0.0,1.0,1.0,4.0,5.0
2,197255,912,0.0,0.0,1.0,4.0,5.0
3,197255,1734,0.0,0.0,3.0,4.0,7.0
4,197255,2254,0.0,0.0,1.0,4.0,5.0
...,...,...,...,...,...,...,...
4980,6515504,10692,1.0,1.0,4.0,4.0,8.0
4981,6601130,10455,0.0,1.0,2.0,4.0,6.0
4982,6610199,10122,0.0,1.0,2.0,4.0,6.0
4983,6610199,10839,0.0,1.0,2.0,4.0,6.0


### 1.10 - Подготовка модуля поддержки группы

In [74]:
ext_dict = {   
      'Ba1 / BB+': 10
    , 'Ba2 / BB': 12
    , 'Baa2 / BBB': 7
    , 'Baa3 / BBB-': 8
}

In [75]:
group_support_cols_dict = module_cols['group_support']

group_support_cols = list(group_support_cols_dict.values())

In [76]:
# group_df = raw_data['ratings_full'][id_cols+list(group_support_cols_dict.keys())].rename(group_support_cols_dict, axis=1)

group_df = filtered_df[['client_id', 'rating_id']].merge(
                            raw_data['ratings_full'][id_cols+list(group_support_cols_dict.keys())]
                        , how='left'
                        , on=['client_id', 'rating_id']
                        )\
                        .rename(group_support_cols_dict, axis=1)

In [77]:
for col in group_df.select_dtypes(object):
    group_df[col] = group_df[col].str.strip()
    group_df[col] = group_df[col].str.replace(' ', '')
    group_df[col] = group_df[col].replace({  'Да'  : 1
                                           , 'Нет' : 0 
                                           , 'Низкое(<25%)' : 0 
                                           , 'Среднее(25%-50%)' : 1
                                           , 'Высокое(>50%)' : 2
                                           })

In [78]:
group_df.fillna({col : 0  for col in ['kgrs3','kgrs4','kgrs5','kgrs6','kgrs7','kgrs8','kgrs9','kgrs10','kgrs11','PD_int']}, inplace=True)

In [79]:
group_df['Ww'] = 0.11 * group_df.kgrs3 + \
                 0.15 * group_df.kgrs4 + \
                 0.14 * group_df.kgrs5 + \
                 0.12 * group_df.kgrs6 + \
                 0.15 * group_df.kgrs7 + \
                 0.11 * group_df.kgrs8 + \
                 0.08 * group_df.kgrs9 + \
                 0.11 * group_df.kgrs10 + \
                 0.08 * group_df.kgrs11 
    
group_df.loc[group_df.kgrs1 == 2, 'Wa'] = 0
group_df.loc[group_df.kgrs1 == 1, 'Wa'] = 0.75
group_df.loc[group_df.kgrs1 == 0, 'Wa'] = 1

group_df.loc[(group_df.kgrs2 == 2) & (group_df.kgrs1 == 0), 'Wa'] = 0.75
group_df.loc[(group_df.kgrs2 == 2) & (group_df.kgrs1 == 1), 'Wa'] = 0

group_df.loc[group_df.Wa.isna() == True, 'Wa'] = 0
group_df['Wgr'] = np.where(0.9 <= group_df.Ww * group_df.Wa, 0.9, group_df.Ww * group_df.Wa)
group_df['PD_ext'] = group_df['PD_ext'].map(ext_dict)
group_df['PDg'] = group_df[['PD_ext', 'PD_int']].max(axis = 1)

group_df.head(4)

,client_id,rating_id,PD_int,PD_ext,kgrs1,kgrs2,kgrs3,kgrs9,kgrs4,kgrs6,kgrs10,kgrs7,kgrs11,kgrs8,kgrs5,Ww,Wa,Wgr,PDg
0,176783,1737,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,197185,1048,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,197255,912,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,197255,1734,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1.11 - Подготовка модуля поддержки государства 

In [80]:
gov_mapping = raw_data['gov_mapping']
new_gov_name_mapping = module_cols['gov_support']

org_gov_cols = list(new_gov_name_mapping.keys())
renamed_gov_features = ['kgs1', 'kgs2', 'kgs3', 'kgs4', 'kgs5']

In [81]:
gov_df = filtered_df[id_cols].merge(
              raw_data['ratings_full'][id_cols + org_gov_cols]
            , how='left'
            , on=id_cols
            )

In [82]:
gov_df.replace(gov_mapping, inplace=True)
gov_df.rename(columns=new_gov_name_mapping, inplace=True)
gov_df.fillna({c: 0 for c in renamed_gov_features}, inplace=True)
gov_df['rate_info'] = gov_df['rate_info'].str.strip()
gov_df.replace({ 'rate_info':{'': None}},inplace=True)

In [83]:
gov_df['kgs_sum'] = gov_df[renamed_gov_features].sum(axis=1)
gov_df['gov_Wgr'] = np.where(0.9 >= gov_df['kgs_sum'], gov_df['kgs_sum'], 0.9)
gov_df['gov_rating'] = gov_df[['internal_rate', 'external_rate']].max(axis = 1)
gov_df.head()

,client_id,rating_id,internal_rate,external_rate,rate_info,kgs1,kgs2,kgs3,kgs4,kgs5,kgs_sum,gov_Wgr,gov_rating
0,176783,1737,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,197185,1048,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,197255,912,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,197255,1734,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,197255,2254,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [84]:
gov_df.isna().sum()

client_id           0
rating_id           0
internal_rate    4985
external_rate    4978
rate_info        4983
kgs1                0
kgs2                0
kgs3                0
kgs4                0
kgs5                0
kgs_sum             0
gov_Wgr             0
gov_rating       4978
dtype: int64

### 1.12 - Подготовка модуля предупреждающих сигналов 

In [85]:
warnings_info = raw_data['warnings'].copy()

In [86]:
no_longer_used_warnings = warnings_info.query('Наличие == "Нет"')['Код интеграции']
no_longer_used_warnings

11     kws13
13     kws15
21    DT0300
39    DT1100
Name: Код интеграции, dtype: object

In [87]:
warnings_info

,№,Показатель,Ухудшение рейтинга,Код интеграции,Наличие
0,1,Наложение кредиторами ареста на расчетные счет...,2,kws1,Есть
1,2,Изменения в составе собственников и / или мене...,2,kws2,Есть
2,3,Нестабильное поведение уполномоченных представ...,2,kws3,Есть
3,4,Наличие негативной информации о деловой репута...,3,kws4,Есть
4,5,Информация о возможном мошенничестве (в частно...,8,kws5,Есть
5,6,Информация о действиях владельцев / уполномоче...,5,kws7,Есть
6,7,Внесение изменений в учредительные документы к...,8,kws8,Есть
7,8,Существенное нарушение условий кредитной докум...,23,kws9,Есть
8,9,"Имеются доказательства мошенничества, по инфор...",25,kws10,Есть
9,10,Существенное нарушение основных операций контр...,5,kws11,Есть


In [88]:
warnings_info = warnings_info[~warnings_info['Код интеграции'].isin(["5_15_dpd", "16_29_dpd"])]
# warnings_info

In [89]:
warning_support_cols = module_cols['warnings']

In [90]:
warnings_df = filtered_df[id_cols].merge(
              raw_data['ratings_full'][id_cols + warning_support_cols]
            , how='left'
            , on=id_cols
            )

In [91]:
warning_support_cols_mapping = {
  'Наложение кредиторами ареста на расчетные счета контрагента в Банке сроком более 7 дней за последние 6 месяцев'                                                                                                : 'g25'
 , 'Изменения в составе собственников и / или менеджмента которые могут негативно повлиять на бизнес информация о наличии серьезных разногласий и конфронтаций между акционерами и их возможной смене'            : 'g26'
 , 'Нестабильное поведение уполномоченных представителей контрагента в частности их уклонение от встреч / переговоров (по телефону электронной почте факсу и т.д.) с представителями Банка по различным причинам' : 'g27'
 , 'Наличие негативной информации о деловой репутации контрагента (Заемщика/Созаемщика/Гаранта/Залогодателя)'                                                                                                     : 'g28'
 , 'Информация о возможном мошенничестве (в частности несоответствия в предоставляемой информации отказ в предоставлении дополнительной информации и т.д.)'                                                       : 'g29'
 , 'Информация о действиях владельцев / уполномоченных лиц контрагента направленных на разделение / продажу бизнеса продажу основных активов которые могут существенно повлиять на деятельность контрагента'      : 'g31'
 , 'Внесение изменений в учредительные документы контрагента без уведомления Банка которые могут повлечь (или повлекли) за собой недействительность (или ничтожность) кредитной сделки'                           : 'g32'
 , 'Существенное нарушение условий кредитной документации/ договора страхования (кроме платежных обязательств)'                                                                                                   : 'g33'
 , 'Имеются доказательства мошенничества по информации правоохранительных органов или службы собственной безопасности которые могут повлечь за собой существенное ухудшение финансового состояния'                : 'g34'
 , 'Существенное нарушение основных операций контрагента (стихийное бедствие реализация экологических рисков рисков техники безопасности и охраны труда арест имущества)'                                         : 'g35'
 , 'Существенное ухудшение финансового положения или другие инциденты произошедшие после отчетной даты и влияющие на возможность контрагента погасить кредит'                                                     : 'g36'
 , 'Просрочка платежа от 5 до 30 календарных дней подряд на момент оценки'                   : 'g37'
 , 'Просрочка платежа от 30 до 60 календарных дней подряд на момент оценки'                  : 'g38'
 , 'Просрочка платежа от  60 до 90 календарных дней подряд на момент оценки'                 : 'g39'
 , 'Наличие просрочек по налоговым выплатам несущественные по сумме и срокам'                : 'g41'
 , 'Наличие просрочек по налоговым выплатам существенные по сумме и срокам'                  : 'g42'
 , 'Наличие просроченной задолженности непрерывной длительностью свыше 90 календарных дней%' : 'g78'
 , 'Отсутствие выручки за одну отчетную дату из пяти последних отчетных дат* либо присутствие более чем одной отчетности (от двух до четырех отчетностей) из пяти последних отчетных дат'                         : 'g50'
 , 'Отсутствие выручки более чем за одну отчетную дату из пяти последних отчетных дат либо присутствие отчетности только на последнюю дату из пяти последних отчетных дат'                                        : 'g51'
 , 'max(case when nvl(k1.attribute_namek2.attribute_name) in (NPL 90+: корпоративный заемщик просрочил выплату суммы обязательств по договору в Банке более чем на 90 днейNPL90+: заемщик просрочил погашение любой задолженности перед Банком более чем на 90 дней)': 'g79'
 , 'Банкротство: Банк инициировал начало судебного процесса по признанию банкротства%заемщик/Контрагент сам начал процесс по признанию себя банкротом.'                                                           : 'g80'
 , '(Резервы: создание специальных (индивидуальных) резервов.Резервы: увеличение норматива резервирования по причине значительного ухудшения качества задолженности/финансовых обязательств Контрагентов по операциям на финансовых рынках.)': 'g81'
 , 'Принудительное взыскание: Банк инициировал процедуру принудительного взыскания в судебном/внесудебном порядке всей суммы обязательств%заемщика/Контрагента или ее части.': 'g82'
 , 'Ликвидация: принятие%заемщиком/Контрагентом решения о ликвидации или прекращении деятельности%'                                                                          : 'g83'
 , 'Реструктуризация: реструктуризация обязательств%.'                                                                                                                       : 'g84'
 , 'Ожидаемая неплатежеспособность: вероятное непогашение%заемщиком задолженности.'                                                                                          : 'g85'
 , 'Отказ Заемщика от погашения Задолженности выраженный в письменной или устной форме.'                                                                                     : 'g86'
 , 'По результатам проверки ГСЗ выявлено отрицательное влияние члена ГСЗ относящегося к черной зоне обосновывающее отнесение Заемщика к черной зоне.'                        : 'g87'
 , 'Принятие Кредитующим подразделением и ПРПА совместного решения о выборе дефолтной стратегии работы с ПА.'                                                                : 'g88'
 , 'Отсутствие любых источников погашения.'                                                                                                                                  : 'g89'
 , 'Наличие производства по делу о несостоятельности (банкротстве) Поручителя / Гаранта / Залогодателя'                                                                      : 'g90'
 , 'Принятие Коллегиальным органом Банка решения обратить взыскание на заложенное имущество во внесудебном порядке.'                                                         : 'g91'
 , 'Причастность к экстремисткой деятельности.'                                                                                                                              : 'g92'
 , 'Обращение Заемщика с предложением о принятии имущества на баланс Банка в счет погашения задолженности.'                                                                  : 'g93'
 , 'Присвоена "Черная зона проблемности".'                                                                                                                                   : 'g94'
 , 'Инициирование Банком возбуждения уголовных дел в отношении Участника кредитной сделки/его руководителей и/или владельцев.'                                               : 'g95'
 , 'Дефолт на финансовых рынках: вероятное неисполнение Контрагентом финансовых обязательств по операциям на финансовых рынках.'                                             : 'g49'
 , 'Списание: принято решение%о списании задолженности или финансового обязательства по операциям на финансовых рынках'                                                      : 'g96'
 , 'Уступка: уступка прав требования по обязательствам%заемщика/Контрагента с финансовыми потерями для Банка.'                                                               : 'g97'
 , 'Дефолт%в участниках Группы: дефолт по обязательствам перед Банком и другими участниками Группы в результате срабатывания любого  из перечисленных%индикаторов дефолта.'  : 'g98'
}

In [92]:
signal_1 = list(raw_data['warnings']['Показатель'].values)
signal_2 = list(warning_support_cols_mapping.keys())

signal_to_signal_mapping = {
    0: 0,   # Наложение ареста на счета
    1: 1,   # Изменения в составе собственников
    2: 2,   # Нестабильное поведение представителей
    3: 3,   # Негативная информация о деловой репутации
    4: 4,   # Информация о возможном мошенничестве
    5: 5,   # Действия по разделению/продаже бизнеса
    6: 6,   # Внесение изменений в учредительные документы
    7: 7,   # Нарушение условий кредитной документации
    8: 8,   # Доказательства мошенничества от правоохранительных органов
    9: 9,   # Нарушение основных операций
    10: 10, # Ухудшение финансового положения
    11: 11, # Просрочка 5-30 дней
    12: 12, # Просрочка 30-60 дней
    13: 13, # Просрочка 60-90 дней
    14: 14, # Несущественные просрочки по налогам
    15: 15, # Существенные просрочки по налогам
    16: 16, # Просроченная задолженность свыше 90 дней
    17: 17, # Отсутствие выручки за одну отчетную дату
    18: 18, # Отсутствие выручки более чем за одну дату
    19: 19, # NPL 90+
    20: 20, # Банкротство
    21: 21, # Резервы
    22: 22, # Принудительное взыскание
    23: 23, # Ликвидация
    24: 24, # Реструктуризация
    25: 25, # Ожидаемая неплатежеспособность
    26: 26, # Отказ от погашения задолженности
    27: 27, # Отрицательное влияние члена ГСЗ
    28: 28, # Решение о дефолтной стратегии
    29: 29, # Отсутствие источников погашения
    30: 30, # Банкротство Поручителя/Гаранта
    31: 31, # Решение об обращении взыскания
    32: 32, # Причастность к экстремистской деятельности
    33: 33, # Предложение о принятии имущества
    34: 34, # Черная зона проблемности
    35: 35, # Возбуждение уголовных дел
    36: 36, # Дефолт на финансовых рынках
    37: 37, # Списание
    38: 38, # Уступка
    39: 39, # Дефолт Единого клиента
}

warnings_info['Показатель2'] = signal_2
warnings_info['Название в коде'] = warnings_info['Показатель2'].map(warning_support_cols_mapping)
code_map = warnings_info[['Название в коде', 'Код интеграции']].set_index('Название в коде')['Код интеграции'].to_dict()
warnings_df.columns = [code_map[col] if col in code_map.keys() else col for col in warnings_df.columns]

In [93]:
warnings_df.drop(no_longer_used_warnings, axis=1, inplace=True)

### 1.13 - Сопостовление релевантных рейтингов

In [94]:
# Find common rating_ids
common_rating_ids = set(quant_df_processed['rating_id']) & set(qual_df_processed['rating_id'])

# Filter all dataframes in place
for df_name in  ['filtered_df', 'qual_df_processed', 'quant_df_processed', 'group_df', 'gov_df', 'warnings_df', 'youngs_df', 'valid_dates']:
    df = globals()[df_name]
    globals()[df_name] = df[df['rating_id'].isin(common_rating_ids)].copy()
    print(f"{df_name} shape: {globals()[df_name].shape}")

filtered_df shape: (4984, 12)
qual_df_processed shape: (4984, 26)
quant_df_processed shape: (4984, 125)
group_df shape: (4984, 19)
gov_df shape: (4984, 13)
warnings_df shape: (4984, 38)
youngs_df shape: (4984, 7)
valid_dates shape: (4984, 4)


### 1.14 - Проверка на заполняемость в финальных датафреймах

In [95]:
def get_missing_pct(df):
    stats = np.round(df.isna().sum()/ df.shape[0] * 100, 2).sort_values(ascending=False)
    stats_only_missing = stats.loc[stats.index[stats>0]].to_frame().rename(columns={0:'missing_pct'})
    return stats_only_missing

In [96]:
for dd_name in ['filtered_df', 'qual_df_processed', 'quant_df_processed', 'group_df', 'gov_df', 'warnings_df', 'youngs_df', 'valid_dates']:
    dd = globals()[dd_name]
    print('---------')
    print(dd_name)
    if get_missing_pct(dd).empty:
        print('No missing values')
    else:
        display(get_missing_pct(dd))

---------
filtered_df
No missing values
---------
qual_df_processed
No missing values
---------
quant_df_processed


,missing_pct
h9,5.24
d26,5.24
h13,5.22
h10,5.22
h3,5.22
h11,5.22
h7,5.22
h5,5.22
h4,5.22
h8,5.22


---------
group_df


,missing_pct
PD_ext,100.00
kgrs1,97.21
kgrs2,97.21


---------
gov_df


,missing_pct
internal_rate,100.00
rate_info,99.96
external_rate,99.86
gov_rating,99.86


---------
warnings_df


,missing_pct
DT0600,8.19
DT0100,8.11
DT0700,8.11


---------
youngs_df
No missing values
---------
valid_dates
No missing values


### 1.15 - Выгрузка данных

In [97]:
filtered_df.to_parquet(filepaths['clean_data'])

# features
qual_df_final.to_parquet(filepaths['qualitative_df'])
quant_df_processed.to_parquet(filepaths['quantitative_df'])

# поддержка группы
group_df.to_parquet(filepaths['group_df_preprocessed'])
# group_df[['rating_id', 'PDg', 'Wgr']].to_parquet(filepaths['group_df_influence'])

# поддержка государства
gov_df.to_parquet(filepaths['gov_df_preprocessed'])
gov_df[['rating_id', 'kgs_sum', 'gov_Wgr', 'gov_rating']].to_parquet(filepaths['gov_df_influence'])

# пред. сигналы
warnings_df.to_parquet(filepaths['warnings_df'])

# для определения молодых компаний
youngs_df.to_parquet(filepaths['youngs_df'])

# валидные даты рейтингов
valid_dates.to_parquet(filepaths['valid_dates'])